In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import openai
import pandas as pd

In [ ]:
%cd /content/drive/MyDrive/자기주도연구

/content/drive/MyDrive/자기주도연구


In [ ]:
!pwd

/content/drive/MyDrive/자기주도연구


In [ ]:
from openai import OpenAI

# GPT API를 사용한 드라마 관련 여부 판별 함수
def classify_comment(comment, context):
  # 비동기 클라이언트 인스턴스 생성
  client = OpenAI(
      api_key = "my_gpt_api_key"
  )

  # 동기 요청
  response = client.chat.completions.create(
        model="gpt-4o-mini",  # 비용 효율성을 위해 GPT-4o mini 사용
        messages=[
            {"role": "system", "content": "You are an assistant that classifies comments as related or unrelated to a specific drama. If a comment is related, return 0. If unrelated, return 1."},
            {"role": "user", "content": f"Context: '{context}'\nComment: '{comment}'\nDoes this comment relate to the context? Respond only with 0 or 1."}
        ]
    )
  # GPT의 응답 내용 추출
  result = response.choices[0].message.content
  return result

In [ ]:
comment = pd.read_csv('데이터 수집/comments_오징어게임2.csv')

In [ ]:
comment

,title,comment
0,오징어게임2,옛날놀이들이 있는데 말?도 그런듯..
1,오징어게임2,한줄평 - 쇼군2가 훨씬 재밌겠다.
2,오징어게임2,오겜2는 개졸작
3,오징어게임2,그냥 시즌 1에서 깔끔하게 끝내던지 아니면 2부에서 1부의 복선을 회수하던지 하지....
4,오징어게임2,&quot;오영일의 정체..&quot;<br>시즌01과 같이 &quot;오일남&qu...
...,...,...
3601,오징어게임2,딸깍
3602,오징어게임2,눈빛 지리네
3603,오징어게임2,와우
3604,오징어게임2,꺅


In [ ]:
comment['relation'] = '0'

# 첫번째 필터링

In [ ]:
# 드라마의 맥락
drama_context = "드라마 오징어게임2를 본 시청자들의 반응 데이터만을 수집해야되. 해당 드라마에 대한 이야기, 배우, 그 밖에 드라마에 대한 이야기여야해. 그리고 중간중간 배우의 별명이 나올 수 있어. 그 별명은 다른 작품에서 해당 배우들이 맡은 배역이거나 팬들이 부르는 별명일 수 있어. 이런 것도 관련있는거고 드라마에서 나타나는 느낌에 대한 내용들은 관련있는 것으로 판단해줘! 아예 다른 드라마에 대한 내용이거나 결방과 같은 드라마 트랜드와는 관련없는 내용이거나 그냥 단순히 재밌다 또는 광고 댓글이면 관련없는 걸로 판단해줘."

# 결과 저장
for i in range(len(comment)):
  comment.loc[i, 'relation'] = classify_comment(comment.iloc[i]['comment'], drama_context)


In [ ]:
for i in range(len(comment)):
  if '0' in comment.loc[i,'relation']: comment.loc[i,'relation'] = 0
  else: comment.loc[i,'relation'] = 1

In [ ]:
print('관련없음 :',comment['relation'].sum(),'/',len(comment))
print('관련있음 :',len(comment)-comment['relation'].sum(),'/',len(comment))

관련없음 : 278 / 511
관련있음 : 233 / 511


# 두번째 필터링
### 첫번째 필터링에서 찾아내지 못한 무관한 댓글을 찾기 위해 필터링 한번 더 진행

In [ ]:
# 드라마의 맥락
drama_context = "드라마 오징어게임2를 본 시청자들의 반응 데이터만을 수집해야되. 해당 드라마에 대한 이야기, 배우, 그 밖에 드라마에 대한 이야기여야해. 그리고 중간중간 배우의 별명이 나올 수 있어. 그 별명은 다른 작품에서 해당 배우들이 맡은 배역이거나 팬들이 부르는 별명일 수 있어. 이런 것도 관련있는거고 드라마에서 나타나는 느낌에 대한 내용들은 관련있는 것으로 판단해줘! 아예 다른 드라마에 대한 내용이거나 결방과 같은 드라마 트랜드와는 관련없는 내용이거나 그냥 단순히 재밌다 또는 광고 댓글이면 관련없는 걸로 판단해줘."

# 결과 저장
for i in range(len(comment)):
  if comment.loc[i,'relation']==1:
    result = classify_comment(comment.iloc[i]['comment'], drama_context)
    if '0' in result: comment.loc[i,'relation'] = 0
    else: comment.loc[i,'relation'] = 1


# 필터링한 정보만 남기기

In [ ]:
for i in range(len(comment)):
  if comment.loc[i,'relation']==1: comment.drop(i,inplace=True)

In [ ]:
comment['relation'].sum()

0

In [ ]:
comment.to_csv('comments_오징어게임2_after_filtering.csv',index=False)